In [1]:
import numpy as np

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.autograd import Variable
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import tensor
import torch.optim as optim

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### CNN classifier

In [43]:
class CNN(nn.Module):
    def __init__(self, activation_fn, drop_out_rate, batch_size, epochs, train_data, val_data, test_data, loss_fn, lr, optimizer):

        # Inherit from nn module
        super(CNN, self).__init__()

        # Define activation function
        self.activation_fn = activation_fn

        # Define drop ratio
        self.drop_out_rate = drop_out_rate

        # Define neural network architecture
        self.nn = nn.Sequential(
                # C1 6@28x28
                nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
                nn.BatchNorm2d(6),
                self.activation_fn(),
                nn.AvgPool2d(kernel_size=2, stride=2),
                
                # C2: 16@10x10
                nn.Dropout(self.drop_out_rate),
                nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
                nn.BatchNorm2d(16),
                self.activation_fn(),
                nn.AvgPool2d(kernel_size=2, stride=2),
                
                # Apply flattening on the output
                nn.Flatten(),
                
                # Dense part
                # L1
                nn.Dropout(self.drop_out_rate),
                nn.Linear(16 * 5 * 5, 120),
                nn.BatchNorm1d(120),
                self.activation_fn(),
                
                # L2
                nn.Dropout(self.drop_out_rate),
                nn.Linear(120, 84),
                nn.BatchNorm1d(84),
                self.activation_fn(),
                
                # L3
                nn.Dropout(self.drop_out_rate),
                nn.Linear(84, 10))
        
        # Define batch size
        self.batch_size = batch_size

        # Define numper of epochs
        self.epochs = epochs

        # Define datasets
        self.training_data = DataLoader(
            train_data, batch_size=self.batch_size, shuffle=True, num_workers=1)
        self.validation_data = DataLoader(
            val_data, batch_size=self.batch_size, shuffle=True, num_workers=1)
        self.test_data = DataLoader(
            test_data, batch_size=len(test_data), shuffle=True, num_workers=1)

        # Define loss function
        self.loss_fn = loss_fn

        # Define learning rate
        self.lr = lr

        # Define optimizer
        self.optimizer = optimizer(self.parameters(), lr=self.lr)

        # Save training progress
        self.loss_history = []
        self.acc_history = []
    
    def forward(self, x):
        logits = self.nn(x)
        return logits
    
    def train_loop(self):
        size = len(self.training_data.dataset)
        for batch, (X, y) in enumerate(self.training_data):

            # Compute prediction and loss
            pred = self.forward(X)
            loss = self.loss_fn(pred, y)

            # Backpropagation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            if batch % 20 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    def val_loop(self):
        size = len(self.validation_data.dataset)
        num_batches = len(self.validation_data)
        test_loss, correct = 0, 0

        with torch.no_grad():
            for X, y in self.validation_data:
                pred = self.forward(X)
                test_loss += self.loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
        test_loss /= num_batches
        correct /= size

        print(
            f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

        # Save it to history
        self.acc_history.append(correct)
        self.loss_history.append(test_loss)

    def fit(self):
        for t in range(self.epochs):
            print(f"Epoch {t+1}\n-------------------------------")
            self.train_loop()
            self.val_loop()
        print("Done!")

    def predict(self, x):
        logits = self.forward(x)
        softmax = nn.Softmax(dim=1)
        return softmax(logits).argmax(1)

    def test(self):
        # Get data
        X, y = next(iter(self.test_data))

        # Predict values
        y_hat = self.predict(X)

        print("Accuracy score for test data")
        print("-"*60)
        print(f"Acc: {accuracy_score(y, y_hat)*100} %")
        print()
        print("Confusion matrix for test data")
        print("-"*60)
        print(confusion_matrix(y, y_hat))

In [44]:
# Load the data
data_train = np.load('data/fashion_train.npy', mmap_mode='r')
data_test = np.load('data/fashion_test.npy', mmap_mode='r')

X_train = data_train[:, :-1]
y_train = data_train[:, -1]

X_test = data_test[:, :-1]
y_test = data_test[:, -1]

# Convert to PyTorch tensors
X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train).long()
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long) 

# Reshape X_train and X_test for convolutional input (assuming 28x28 images)
X_train_tensor = X_train_tensor.view(-1, 1, 28, 28)  # Adjust dimensions based on your actual image size
X_test_tensor = X_test_tensor.view(-1, 1, 28, 28)  # Adjust dimensions based on your actual image size


# Create TensorDataset
data_train = TensorDataset(X_train_tensor, y_train_tensor)

In [45]:
# Split training into training and validation
g_cpu = torch.Generator()
g_cpu.manual_seed(3)
data_train, data_val = torch.utils.data.random_split(data_train, [8000, 2000])

In [46]:
# Initialize model
input = {'activation_fn': nn.Sigmoid,
         'drop_out_rate': 0.,
         'batch_size': 100, 
         'epochs': 10, 
         'train_data': data_train, 
         'val_data': data_val, 
         'test_data': data_test, 
         'loss_fn': nn.CrossEntropyLoss(), 
         'lr': 0.1, 
         'optimizer': torch.optim.Adam 
         }
model = CNN(input['activation_fn'], 
            input['drop_out_rate'], 
            input['batch_size'], 
            input['epochs'], 
            input['train_data'], 
            input['val_data'], 
            input['test_data'], 
            input['loss_fn'], 
            input['lr'], 
            input['optimizer'])

In [47]:
model.fit()

Epoch 1
-------------------------------
loss: 2.151961  [    0/ 8000]
loss: 1.117417  [ 2000/ 8000]
loss: 0.505964  [ 4000/ 8000]
loss: 0.642222  [ 6000/ 8000]
Test Error: 
 Accuracy: 74.0%, Avg loss: 0.642387 

Epoch 2
-------------------------------
loss: 0.404509  [    0/ 8000]
loss: 0.534760  [ 2000/ 8000]
loss: 0.565140  [ 4000/ 8000]
loss: 0.581934  [ 6000/ 8000]
Test Error: 
 Accuracy: 81.0%, Avg loss: 0.510328 

Epoch 3
-------------------------------
loss: 0.445292  [    0/ 8000]
loss: 0.556170  [ 2000/ 8000]
loss: 0.427615  [ 4000/ 8000]
loss: 0.492390  [ 6000/ 8000]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.471627 

Epoch 4
-------------------------------
loss: 0.348552  [    0/ 8000]
loss: 0.378745  [ 2000/ 8000]
loss: 0.456573  [ 4000/ 8000]
loss: 0.444900  [ 6000/ 8000]
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.410467 

Epoch 5
-------------------------------
loss: 0.400454  [    0/ 8000]
loss: 0.428064  [ 2000/ 8000]
loss: 0.380284  [ 4000/ 8000]
loss: 0.463567  [ 6000